In [6]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain_openai import ChatOpenAI

In [8]:
from dotenv import load_dotenv
load_dotenv()

True

In [9]:
KEY = os.getenv("OPENAI_API_KEY")

In [11]:
llm = ChatOpenAI(api_key=KEY, model_name = "gpt-4o", temperature = 0.3)

In [12]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [13]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "option a",
            "b": "option b",
            "c": "option c",
            "d": "option d",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "option a",
            "b": "option b",
            "c": "option c",
            "d": "option d",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "option a",
            "b": "option b",
            "c": "option c",
            "d": "option d",
        },
        "correct": "correct answer",
    },
}

In [14]:
TEMPLATE = """
Text: {text}
You are an expert MCQ test maker. Given the above text, it is your job to \n
create a quiz of {number} multiple choice questions for {subject} students in a {tone} tone.
Make sure the questions are not repeated and check to make sure that all the questions can be solved with information \n
from the text. Make sure to format your questions like the RESPONSE_JSON below.
Ensure that you make {number} questions.
### RESPONSE_JSON:
{response_json}
"""

In [15]:
quiz_generation_prompt = PromptTemplate(
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [16]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)

In [17]:
TEMPLATE2 = """
You are an English writer who is very proficient in the English langauge, especically grammar. Given a \n
multiple choice quiz for {subject} students, you need to evaluate the complexity of the questions and provide a \n
complexity analysis of the quiz. Only use at max 50 words for the ocmplexity analysis. If the quiz is not on par with \n
the cognitive and analytical abilities of the students, update the quiz questions which need to be changed so that the tone \n
is appropriate for the students' cogntive level.
Quiz MCQs:
{quiz}

Check from an expert English writer of the above quiz:
"""

In [18]:
quiz_evaluation_prompt = PromptTemplate(
    input_variables = ["subject", "quiz"],
    template = TEMPLATE2
)

In [19]:
review_chain = LLMChain(llm = llm, prompt = quiz_evaluation_prompt, output_key = "review", verbose = True)

In [20]:
generate_evaluate_quiz = SequentialChain(
    chains = [quiz_chain, review_chain],
    input_variables = ["text", "number", "subject", "tone", "response_json"],
    output_variables = ["quiz", "review"],
    verbose = True,
)

In [ ]:
file_path = r"/Users/suhaspalawala/Documents/GitHub/mcq-generator/data.txt"

In [ ]:
with open(file_path, 'r') as file:
    TEXT = file.read()